# COMP7043 Deep Learning and its Applications - Assignment 1
-------

In the programming part of assignment 1, you are going to train and experiment convolutional neural network on the CIFAR 100 dataset. The network is a Simple convolutional neural network defined with the `nn.Module` by yourself.

When doing the homework, please read the description first then write your code in the pre-defined area. **Please import the needed package and ensure your code can run without modification under the Kaggle or Colab environment.**

**Submit your notebook in a `*.ipynb` file.**

## CIFAR-100
The CIFAR-100 dataset is a commonly used benchmark dataset in the field of computer vision. It is an extension of the CIFAR-10 dataset and consists of 60,000 32x32 color images belonging to 100 different classes. Each class contains 600 images. The dataset is divided into 50,000 training images and 10,000 test images

The classes in CIFAR-100 cover a wide range of objects and concepts, including animals, vehicles, household items, insects, plants, and more. Some examples of classes in CIFAR-100 are "apple," "bee," "bus," "dolphin," "oak_tree," "rose," "sunflower," and "whale."

In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

d:\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define a Transformation function that do following steps:
# 1. RandomHorizontalFlip
# 2. RandomRotation with any degree
# 3. ToTensor
# 4. Normalize with Mean (0.5071, 0.4867, 0.4408), Std (0.2675, 0.2565, 0.2761)

from torchvision import transforms

transform_train = transforms.Compose([
    # Place your code here
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-180, 180)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

In [3]:
# Load the CIFAR-100 dataset with torchvision.datasets module and uses transform_train for transformation

# Place your code below
from torchvision import datasets

dataset = datasets.CIFAR100(root="D:\\desktop\\torch tutorial\\data", train=True, download=True, transform=transform_train)


Files already downloaded and verified


In [4]:
# Use train_test_split function from the sklearn.model_selection to split the CIFAR dataset into train and test set with a ratio of 7:3
# Then create two dataloader for the two sets and set the batch_size to 32, shuffle the training set but not the test set

# Place your code below
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
 
# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=42)

# Define batch size
batch_size = 32
 
# Create data loaders for training, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [5]:
# Create a simple convolutional neural network with nn.Module,
# the detailed structure is not restricted, but you need to include following layers in your network:
# 1. nn.Linear
# 2. nn.Conv2d
# 3. Any Pooling Layer https://pytorch.org/docs/stable/nn.html#pooling-layers
# 4. Any Normalization Layer https://pytorch.org/docs/stable/nn.html#normalization-layers
# 5. nn.Dropout
# 6. Any Activation Layer https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity

import torch.nn as nn

# Place your code below
class MyConvNet(nn.Module):
    def __init__(self, in_channels, out_classes=100):
        super(MyConvNet, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3,  16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Normalization layer
        self.norm = nn.BatchNorm2d(64)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 16 * 16, 4096)
        self.fc2 = nn.Linear(4096, out_classes)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.2)
        
        # Activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        
        x = self.relu(self.conv2(x))
        X = self.pool(x)

        x = self.relu(self.conv3(x))
        X = self.pool(x)

        x = self.norm(x)

        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.relu(x)

        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create an instance of the ConvNet
model = MyConvNet(in_channels=3, out_classes=100)

# Print model architecture
print(model)

# Calculate the parameter count
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")

MyConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16384, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=100, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
)
Total parameters: 67546372


In [6]:
# Instantiate the network and print out the structure with summary module from torchsummary package

# Place your code below
from torchsummary import summary

# Instantiate the network
model = MyConvNet(in_channels=3, out_classes=100)
model = model.to(device)

# Print the model summary
summary(model,(3,32,32))  # Provide an example input size

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
         MaxPool2d-3           [-1, 16, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]           4,640
              ReLU-5           [-1, 32, 16, 16]               0
         MaxPool2d-6             [-1, 32, 8, 8]               0
            Conv2d-7           [-1, 64, 16, 16]          18,496
              ReLU-8           [-1, 64, 16, 16]               0
         MaxPool2d-9             [-1, 64, 8, 8]               0
      BatchNorm2d-10           [-1, 64, 16, 16]             128
           Linear-11                 [-1, 4096]      67,112,960
             ReLU-12                 [-1, 4096]               0
          Dropout-13                 [-1, 4096]               0
           Linear-14                  [

In [7]:
# Instantiate a loss function and optimizer for your model,
# you can use whatever loss function and optimizer that is provided by torch.

# Place your code below
import torch.optim as optim

# Instantiate the loss function
criterion = nn.CrossEntropyLoss()

# Instantiate the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Move all your model and loss function to the device that the model going to train.

# Place your code below
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Move the loss function to the device
criterion.to(device)


CrossEntropyLoss()

In [15]:
# Complete the following training iteration
# then train your model on the training set for 5 iteration.

import torch.nn.functional as F

def metric(batch_predictions, batch_labels):
    # Convert the predictions and labels to numpy arrays
    _, predicted_labels = torch.max(batch_predictions, dim=1)
    correct_predictions = (predicted_labels == batch_labels).sum().item()
    # Calculate the prediction error
    accuracy = correct_predictions / len(batch_labels)
    return accuracy

num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_metric = 0.0
    batch_count = 0
    
################## Complete TODO below ##################
    # Iterate over the batches in the dataloader
    for batch in train_loader:
        # TODO: sent batch data to device and clear gradients from previous iteration
        
        batch_data, batch_labels = batch[0], batch[1]
        batch_data = batch_data.to(device)  # Sent batch data to the device (e.g., GPU)
        batch_labels = batch_labels.to(device)  # Sent batch labels to the device
        # print(batch_data.size())
        optimizer.zero_grad()

        # Forward pass
        batch_predictions = model(batch_data)
        # print(batch_predictions)
        
        # TODO: Calculate loss with the loss function
        loss = criterion(batch_predictions, batch_labels)
        epoch_loss += loss.item()
 
        # TODO: Backward pass
        loss.backward()
        optimizer.step()  # Update model parameters

        # Calculate and record metrics
        batch_metric = metric(batch_predictions, batch_labels)
        epoch_metric += batch_metric
 
        batch_count += 1
################## End of TODO part ##################
    # Calculate average loss and metric for the epoch
    avg_epoch_loss = epoch_loss / batch_count
    avg_epoch_metric = epoch_metric / batch_count
 
    # Print or log the metrics for analysis
    print(f"Epoch {epoch+1} - Loss: {avg_epoch_loss:.4f} - Acc: {avg_epoch_metric:.4f}")

Epoch 1 - Loss: 0.1452 - Acc: 0.9624
Epoch 2 - Loss: 0.1508 - Acc: 0.9612
Epoch 3 - Loss: 0.1491 - Acc: 0.9621
Epoch 4 - Loss: 0.1383 - Acc: 0.9647
Epoch 5 - Loss: 0.1167 - Acc: 0.9684
Epoch 6 - Loss: 0.1249 - Acc: 0.9678
Epoch 7 - Loss: 0.1229 - Acc: 0.9689
Epoch 8 - Loss: 0.1254 - Acc: 0.9691
Epoch 9 - Loss: 0.1389 - Acc: 0.9660
Epoch 10 - Loss: 0.1192 - Acc: 0.9703


In [16]:
# Mimic the training iteration and complete the code for test phase (TODO),
# the test phase should print out the accuracy score for your model on the test set.

model.eval()  # Set the model in evaluation mode
epoch_metric = 0.0
batch_count = 0

################## Complete TODO below ##################

for batch in test_loader:
    # TODO: sent batch data to device
    inputs = batch[0].to(device)
    labels = batch[1].to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(inputs)

    # TODO: Forward pass through the model
    preds = torch.argmax(outputs, dim=1)
    correct = (preds == labels).sum().item()
    
    # Calculate and record metrics
    batch_metric = correct / labels.size(0) # TODO: Use the metric to calculate the accuracy
    epoch_metric += batch_metric
    batch_count += 1
    
################## End of TODO part ##################

avg_epoch_metric = epoch_metric / batch_count
print(f"Test Accuarcy: {avg_epoch_metric:.4f}")

Test Accuarcy: 0.1171
